## Imports

Load the standard library helpers and the Collinear client used below.


In [ ]:
import json
from pathlib import Path

import nest_asyncio
from collinear.client import Client

# Necessary to run in a Jupyter notebook
nest_asyncio.apply()


## Load Configuration

Read the simulation settings and TraitMix configuration from the `configs/` directory.


In [ ]:
CONFIG_DIR = Path('configs')
SIMULATION_CONFIG_FILE = CONFIG_DIR / 'simulation_config.json'
config_data = json.loads(SIMULATION_CONFIG_FILE.read_text())

traitmix_config_value = config_data.get('traitmix_config_file') or 'traitmix_config_airline.json'
traitmix_candidate = Path(traitmix_config_value)
if not traitmix_candidate.is_absolute():
    traitmix_candidate = CONFIG_DIR / traitmix_candidate.name
TRAITMIX_CONFIG_FILE = traitmix_candidate
TRAITMIX_CONFIG = json.loads(TRAITMIX_CONFIG_FILE.read_text())
TRAITMIX_TASKS = list(TRAITMIX_CONFIG.get('tasks', []))

client_settings = config_data.get('client', {}) or {}
CLIENT_ASSISTANT_MODEL_URL = client_settings.get('assistant_model_url', 'https://api.openai.com/v1')
CLIENT_ASSISTANT_MODEL_API_KEY = client_settings.get('assistant_model_api_key')
CLIENT_ASSISTANT_MODEL_NAME = client_settings.get('assistant_model_name', 'gpt-4o-mini')
CLIENT_COLLINEAR_API_KEY = client_settings.get('collinear_api_key', 'demo-001')
CLIENT_TIMEOUT = float(client_settings.get('timeout', 120))
CLIENT_MAX_RETRIES = int(client_settings.get('max_retries', 3))
CLIENT_RATE_LIMIT_RETRIES = int(client_settings.get('rate_limit_retries', 6))

simulate_settings = config_data.get('simulate', {}) or {}
SIM_SAMPLES = simulate_settings.get('k', 1)
SIM_EXCHANGES = simulate_settings.get('num_exchanges', 2)
SIM_DELAY = simulate_settings.get('batch_delay', 0.2)
SIM_TRAITMIX_TEMPERATURE = simulate_settings.get('traitmix_temperature', 0.7)
SIM_TRAITMIX_MAX_TOKENS = simulate_settings.get('traitmix_max_tokens', 256)
SIM_MIX_TRAITS = bool(simulate_settings.get('mix_traits', False))
SIM_MAX_CONCURRENCY = int(simulate_settings.get('max_concurrency', 8))

assess_settings = config_data.get('assess', {}) or {}
ASSESS_JUDGE_MODEL_URL = assess_settings.get('judge_model_url')
ASSESS_JUDGE_MODEL_API_KEY = assess_settings.get('judge_model_api_key')
ASSESS_JUDGE_MODEL_NAME = assess_settings.get('judge_model_name')
ASSESS_TEMPERATURE = assess_settings.get('temperature', 0.0)
ASSESS_MAX_TOKENS = assess_settings.get('max_tokens', 512)

print(f'Loaded simulation: {SIMULATION_CONFIG_FILE} | traitmix: {TRAITMIX_CONFIG_FILE} | tasks: {TRAITMIX_TASKS or "<none>"}')


## Initialize Client

Create the Collinear client using the loaded credentials and runtime options.


In [ ]:
if not CLIENT_ASSISTANT_MODEL_API_KEY:
    raise RuntimeError('assistant_model_api_key must be set in configs/simulation_config.json')

client = Client(
    assistant_model_url=CLIENT_ASSISTANT_MODEL_URL,
    assistant_model_api_key=CLIENT_ASSISTANT_MODEL_API_KEY,
    assistant_model_name=CLIENT_ASSISTANT_MODEL_NAME,
    collinear_api_key=CLIENT_COLLINEAR_API_KEY,
    timeout=CLIENT_TIMEOUT,
    max_retries=CLIENT_MAX_RETRIES,
    rate_limit_retries=CLIENT_RATE_LIMIT_RETRIES,
)


## Run Simulations

Generate synthetic conversations for the selected TraitMix configuration.


In [ ]:
simulations = client.simulate(
    traitmix_config=TRAITMIX_CONFIG,
    k=SIM_SAMPLES,
    num_exchanges=SIM_EXCHANGES,
    batch_delay=SIM_DELAY,
    traitmix_temperature=SIM_TRAITMIX_TEMPERATURE,
    traitmix_max_tokens=SIM_TRAITMIX_MAX_TOKENS,
    mix_traits=SIM_MIX_TRAITS,
    max_concurrency=SIM_MAX_CONCURRENCY,
)

for i, sim in enumerate(simulations, start=1):
    p = getattr(sim, 'traitmix', None)
    if p is not None:
        print(
            "TraitMix Persona:\n"
            f"age={p.age}\n"
            f"gender={p.gender}\n"
            f"occupation={p.occupation}\n"
            f"intent={p.intent}\n"
            f"trait={p.trait}" if p.trait else f"traits={p.traits}"
        )
    print("---")
    for msg in sim.conv_prefix:
        role = str(msg.get('role', ''))
        content = str(msg.get('content', ''))
        if content:
            print(f"{role}: {content}")
    print(f"assistant: {sim.response}")
    print()
print('All simulations complete')


## Assess Conversations

Score the generated conversations with the local assessment helper.


In [ ]:
result = client.assess(
    dataset=simulations,
    judge_model_url=ASSESS_JUDGE_MODEL_URL,
    judge_model_api_key=ASSESS_JUDGE_MODEL_API_KEY,
    judge_model_name=ASSESS_JUDGE_MODEL_NAME,
    temperature=ASSESS_TEMPERATURE,
    max_tokens=ASSESS_MAX_TOKENS,
)
print(f"Assessment: {result.message or '<no message>'}")
for scores_map in result.evaluation_result:
    for scores in scores_map.values():
        print(f"  Score: {scores.score}")
        print(f"  Rationale: {scores.rationale}")
